# BERTrend quickstart
The purpose of this notebook is to complement the existing demos available in the directory `bertrend/demos` with some code examples that explain how to integrate BERTrend with your application code.

In [1]:

%load_ext autoreload
%autoreload 2

## BERTrend installation

In [2]:
from pathlib import Path
import pandas as pd
from pandas import Timestamp
from IPython.display import display
from loguru import logger
import os

from bertrend import DATA_PATH
from bertrend.BERTrend import BERTrend
from bertrend import MODELS_DIR
from bertrend.utils.data_loading import load_data, split_data, TEXT_COLUMN
from bertrend.services.embedding_service import EmbeddingService
from bertrend.BERTopicModel import BERTopicModel
from bertrend.topic_analysis.topic_description import generate_topic_description
from bertrend.trend_analysis.weak_signals import analyze_signal


In [3]:
#!pip install bertrend

### Configuration of topic models

In [3]:
# Topic model with default parameters - each parameter of BERTopic can be modified from the constructor or can be read from a configuration file
# overrides the default config to use English
config = '''
# Default configuration file to be used for topic model

# Global parameters
[global]
language = "English"

# BERTopic parameters: https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.__init__
[bertopic_model]
top_n_words = 10
verbose = true
representation_model = ["MaximalMarginalRelevance"] # KeyBERTInspired, OpenAI
zeroshot_topic_list = []
zeroshot_min_similarity = 0

# UMAP parameters: https://umap-learn.readthedocs.io/en/latest/api.html
[umap_model]
n_neighbors = 5
n_components = 5
min_dist = 0.0
metric = "cosine"
random_state = 42

# HDBSCAN parameters: https://hdbscan.readthedocs.io/en/latest/api.html
[hdbscan_model]
min_cluster_size = 5
min_samples = 5
metric = "euclidean"
cluster_selection_method = "eom"
prediction_data = true

# CountVectorizer: https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
[vectorizer_model]
ngram_range = [1, 1]
stop_words = true # If true, will check `language` parameter and load associated stopwords file
min_df = 2

# ClassTfidfTransformer: https://maartengr.github.io/BERTopic/api/ctfidf.html
[ctfidf_model]
bm25_weighting = false
reduce_frequent_words = true

# MaximalMarginalRelevance: https://maartengr.github.io/BERTopic/api/representation/mmr.html
[mmr_model]
diversity = 0.3

# Reduce outliers: https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.reduce_outliers
[reduce_outliers]
strategy = "c-tf-idf"
'''

topic_model = BERTopicModel(config)

In [ ]:
# The TopicModel class is mainly a wrapper around BERTopic and can be used as-is, for example for a first analysis of data (without considering evolving trends, but this is not mandatory at all)


## Using BERTrend for retrospective analysis

### Instantiation of BERTrend


In the case of a **retrospective trend analysis** task, the goal is to identify and evaluate patterns or changes over time within a dataset, allowing for insights into historical performance, behaviors, or events that can inform future decision-making and strategy development.

In this context, the general principle consists in splitting the past data into different time slices. Then each dataset is used to train a separate topic models. Each topic model description corresponding to the older data slice is merged with the next one and decay factors are applied. This allows to have a vision of topic evolution over time

In [4]:
# Basic creation of the object and parametrization
# BERTrend uses several topic models; therefore, it is necessary to pass a topic_model object as a reference
bertrend = BERTrend(topic_model=topic_model)

### 1. Gather historical data to be analyzed


In [5]:
# Here some Trump tweets from: https://github.com/MarkHershey/CompleteTrumpTweetsArchive/blob/master/data/realDonaldTrump_in_office.csv
#!wget "https://raw.githubusercontent.com/MarkHershey/CompleteTrumpTweetsArchive/refs/heads/master/data/realDonaldTrump_in_office.csv"
df = pd.read_csv("realDonaldTrump_in_office.csv",  sep=',',quotechar='"', skipinitialspace=True)
# BERTrend expects specific data format
df = df.rename(columns={'Time': 'timestamp', 'Tweet URL': 'url', "Tweet Text": "text"})
df["source"]=df["ID"]
df["document_id"] = df.index
df.reset_index(inplace=True, drop=True)
df.head(5)

,ID,timestamp,url,text,source,document_id
0,@realDonaldTrump,2017-01-20 06:31,https://twitter.com/realDonaldTrump/status/822...,It all begins today! I will see you at 11:00 A...,@realDonaldTrump,0
1,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/822...,Today we are not merely transferring power fro...,@realDonaldTrump,1
2,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/822...,"power from Washington, D.C. and giving it back...",@realDonaldTrump,2
3,@realDonaldTrump,2017-01-20 11:52,https://twitter.com/realDonaldTrump/status/822...,What truly matters is not which party controls...,@realDonaldTrump,3
4,@realDonaldTrump,2017-01-20 11:53,https://twitter.com/realDonaldTrump/status/822...,"January 20th 2017, will be remembered as the d...",@realDonaldTrump,4


In [ ]:
df.index

### 2. Embed data

In [8]:
# Selection of a subset of data
df = df.head(1000)
client_secret = "bd76aa472dd91aed4a56bf1935dbb802583c119824380d8567086579c0ef3324"
#embedding_service_cfg = {"local": False, "url":"https://localhost:6464"}
embedding_service_cfg = {"local": False, "url":"https://10.132.5.44:6464", "client_secret":client_secret}


embedding_service = EmbeddingService(**embedding_service_cfg)
embeddings, token_strings, token_embeddings = embedding_service.embed(
                texts=df["text"],
            )

2025-03-31 14:29:33.268 | DEBUG    | bertrend.services.embedding_client:get_api_model_name:49 - Model name: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 14:29:33.377 | DEBUG    | bertrend.services.embedding_client:get_num_workers:65 - Number of workers: 2
2025-03-31 14:29:33.381 | DEBUG    | bertrend.services.embedding_service:_remote_embed_documents:207 - Computing embeddings...
2025-03-31 14:29:33.383 | DEBUG    | bertrend.services.embedding_client:embed_documents:124 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 14:29:33.384 | DEBUG    | bertrend.services.embedding_client:embed_documents:128 - Computing embeddings on 1000 documents using (1) batches...
2025-03-31 14:29:35.240 | DEBUG    | bertrend.services.embedding_client:embed_batch:94 - Computing embeddings...
2025-03-31 14:30:03.776 | DEBUG    | bertrend.services.embedding_client:embed_batch:103 - Computing embeddings done for batch


In [12]:
embedding_model_name = embedding_service.embedding_model_name


### 3. Split the data into time slices

This can be done manually for some reason or can be done automatically based on a specified time granularity

In [9]:
from bertrend.utils.data_loading import group_by_days, load_data

day_granularity = 30
grouped_data = group_by_days(df=df, day_granularity=day_granularity)

In [10]:
# Number of sliced data
len(grouped_data)

6

### 4. Train topic models

In [13]:
bertrend.train_topic_models(grouped_data=grouped_data, embedding_model=embedding_model_name, embeddings=embeddings)

2025-03-31 14:30:56.334 | INFO     | bertrend.BERTrend:train_topic_models:240 - Training topic model 1/6...
2025-03-31 14:30:56.336 | DEBUG    | bertrend.BERTrend:_train_by_period:149 - Processing period: 2017-01-20 00:00:00
2025-03-31 14:30:56.336 | DEBUG    | bertrend.BERTrend:_train_by_period:150 - Number of documents: 184
2025-03-31 14:30:56.338 | DEBUG    | bertrend.BERTrend:_train_by_period:152 - Creating topic model...
2025-03-31 14:30:56.340 | DEBUG    | bertrend.BERTopicModel:fit:230 - 	Initializing BERTopic model
2025-03-31 14:30:56.342 | SUCCESS  | bertrend.BERTopicModel:fit:240 - 	BERTopic model instance created successfully
2025-03-31 14:30:56.348 | DEBUG    | bertrend.BERTopicModel:fit:242 - 	Fitting BERTopic model
2025-03-31 14:30:56,350 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-31 14:31:02,184 - BERTopic - Dimensionality - Completed ✓
2025-03-31 14:31:02,185 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03

### 5. (Optional) Save trained_models

In [14]:
bertrend.save_model()

2025-03-31 14:31:19.783 | INFO     | bertrend.BERTrend:save_model:623 - BERTrend model saved to: /home/jerome/dev/cache/models


### 7. Calculate signal popularity

In [16]:
bertrend.calculate_signal_popularity()

In [18]:
window_size = 30

# List of strong and weak signals over time
for ts in bertrend.doc_groups.keys():
    print(ts)
    noise_topics_df, weak_signal_topics_df, strong_signal_topics_df = bertrend.classify_signals(window_size, ts)
    if not weak_signal_topics_df.empty:
        print("Weak signals")
        display(weak_signal_topics_df[["Topic","Representation"]].head(5))
    if not strong_signal_topics_df.empty:
        print("Strong signals")
        display(strong_signal_topics_df[["Topic","Representation"]].head(5))
    print()


2017-01-20 00:00:00
Strong signals


,Topic,Representation
0,0,will_healthcare_disaster_together_coming_getti...
1,1,great_at_to_in_rt_you_the_of_meeting_again



2017-02-19 00:00:00
Weak signals


,Topic,Representation
0,0,win_republicans_now_in_for_your_immigration_wh...
1,1,our_today_jobs_american_to_you_great_it_we_of



2017-03-21 00:00:00
Weak signals


...

,Topic,Representation
0,0,fbi_why_dem_cia_give_asked_her_were_its_away
1,1,realdonaldtrump_potus_like_rt_an_address_end_f...
2,3,obama_meddling_election_nothing_2016_russia_ru...
3,7,north_trade_this_south_meeting_uswomensopen_de...
4,9,democrats_healthcare_they_would_dems_work_as_s...



2017-06-19 00:00:00
Weak signals


,Topic,Representation
0,3,obama_meddling_election_nothing_2016_russia_ru...
1,7,north_trade_this_south_meeting_uswomensopen_de...
2,9,democrats_healthcare_they_would_dems_work_as_s...
3,12,georgia_gop_foxnews_you_thank_congressional_ha...
4,13,market_new_jobs_years_like_really_deal_another...


Strong signals


,Topic,Representation
0,0,fbi_why_dem_cia_give_asked_her_were_its_away
1,1,realdonaldtrump_potus_like_rt_an_address_end_f...


In [22]:
# selection of one particular timestamp to look at
selected_timestamp = Timestamp('2017-04-20 00:00:00')
selected_topic_model = bertrend.restore_topic_model(selected_timestamp)


2025-03-31 14:37:58,544 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


### Get topic description


In [23]:
desc = generate_topic_description(topic_model=selected_topic_model, topic_number=5, filtered_docs=df, language_code="en")


2025-03-31 14:38:09.019 | DEBUG    | bertrend.llm_utils.openai_client:generate_from_history:128 - API returned: ChatCompletion(id='chatcmpl-BH8gxgz5wTS99j3pJavnMf6dssRQn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "title": "China-US Trade Relations: Future Deal Implications",\n  "description": "This theme explores the evolving dynamics of trade relations between China and the United States, particularly in light of recent political changes and leadership transitions. The focus is on how potential trade deals may impact economic policies, bilateral relations, and global markets. As both nations navigate their respective interests, the implications of these negotiations could reshape the landscape of international trade. The discussion also considers the broader context of power transfer within the U.S. government and its influence on foreign policy, especially regarding economic partnerships with major global players like C

In [24]:
desc["title"]

'China-US Trade Relations: Future Deal Implications'

In [25]:
desc["description"]

'This theme explores the evolving dynamics of trade relations between China and the United States, particularly in light of recent political changes and leadership transitions. The focus is on how potential trade deals may impact economic policies, bilateral relations, and global markets. As both nations navigate their respective interests, the implications of these negotiations could reshape the landscape of international trade. The discussion also considers the broader context of power transfer within the U.S. government and its influence on foreign policy, especially regarding economic partnerships with major global players like China.'

### Get topic analysis

In [28]:
summary, analysis, formatted_html = analyze_signal(bertrend, 1, selected_timestamp)

2025-03-31 14:38:45.956 | DEBUG    | bertrend.trend_analysis.weak_signals:analyze_signal:368 - First prompt - generate summary
2025-03-31 14:39:02.091 | DEBUG    | bertrend.llm_utils.openai_client:generate_from_history:128 - API returned: ChatCompletion(id='chatcmpl-BH8ha6VQmNSv23vj9zXrwWvv0a6qz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## A New Era of American Leadership\n### Date: January 20, 2017\n### Key Developments\n- Donald Trump’s inauguration marked a shift in power dynamics, emphasizing a government "for the people."\n- The slogan "America First" was introduced, focusing on job creation, border control, and economic revitalization.\n- A commitment to "Buy American & Hire American" was established as a core principle of the administration.\n\n### Analysis\nThe inauguration of Donald Trump signified a pivotal moment in American politics, characterized by a populist approach aimed at returning power to the citizens. Th

In [29]:
from IPython.display import display, HTML
display(HTML(formatted_html))

## Using BERTrend for prospective analysis

In the case of a **prospective trend analysis task**, the goal is to **forecast future** developments or outcomes based on current data and trends, enabling organizations to make informed decisions, allocate resources effectively, and strategize for upcoming challenges or opportunities.


In this example, we are going to simulate a prospective task:
- we simulate new data coming in
- for each new data, we will compute the new topic model, merge it to previous one and detect at each iteration strong and weak signals


In [39]:
MY_DATA_DIR = Path("/DSIA/nlp/bertrend/data") / "feeds/feed_sobriete"

input_data = [
    MY_DATA_DIR / "2024-12-30_feed_sobriete.jsonl",
    MY_DATA_DIR / "2025-01-06_feed_sobriete.jsonl",
    MY_DATA_DIR / "2025-01-20_feed_sobriete.jsonl",
]

window_size = 7

In [40]:
embedding_service_cfg = {"local": False, "url":"https://10.132.5.44:6464", "client_secret":client_secret}

embedding_service = EmbeddingService(**embedding_service_cfg)
embedding_model_name = embedding_service.embedding_model_name

2025-03-31 14:53:49.611 | DEBUG    | bertrend.services.embedding_client:get_api_model_name:49 - Model name: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 14:53:49.711 | DEBUG    | bertrend.services.embedding_client:get_num_workers:65 - Number of workers: 2


In [41]:
BERTREND_MODELS_PATH = MODELS_DIR / "sobriete_models"

In [52]:
def process_new_data(data_slice_path: Path, timestamp: pd.Timestamp):
    logger.debug(f"Processing new data: {data_slice_path}")

    # Restore previous models
    try:
        bertrend = BERTrend.restore_model(BERTREND_MODELS_PATH)
    except:
        logger.warning("Cannot restore previous models, creating new one")
        bertrend = BERTrend(topic_model=BERTopicModel())

    # Read data
    df = load_data(data_slice_path, language="French")
    df = split_data(df)
    text = df[TEXT_COLUMN]

    # Embed new data
    embeddings, token_strings, token_embeddings = embedding_service.embed(
                texts=text,
    )

    # Create topic model for new data
    bertrend.train_topic_models({timestamp: df}, embeddings=embeddings, embedding_model=embedding_model_name)
    
    logger.info(f"BERTrend processed {len(bertrend.doc_groups)} time periods")
    
    # Save models
    bertrend.save_model(models_path=BERTREND_MODELS_PATH)

    
    if len(bertrend.doc_groups)<2:
        return None
        
    # Compute popularities
    bertrend.calculate_signal_popularity()
    
    # classify last signals
    noise_topics_df, weak_signal_topics_df, strong_signal_topics_df = bertrend.classify_signals(window_size, timestamp)
    # TODO: save dfs

    if weak_signal_topics_df.empty:
        return None
        
    wt = weak_signal_topics_df['Topic']
    logger.info(f"Weak topics: {wt}")
    wt_list = []
    for topic in wt:
        topic_model = bertrend.restore_topic_model(timestamp)
        desc = generate_topic_description(topic_model=topic_model, topic_number=topic, filtered_docs=df, language_code="fr")
        wt_list.append({"timestamp": timestamp, "topic": topic, "title": desc["title"], "description": desc["description"]})

    return pd.DataFrame(wt_list)


2025-03-31 15:27:57.985 | INFO     | bertrend.BERTrend:restore_model:629 - Loading BERTrend model from: /home/jerome/dev/cache/models/sobriete_models


3

In [53]:
for data_file in input_data:
    timestamp = pd.Timestamp(data_file.name.split('_')[0])
    display(process_new_data(data_file, timestamp))

2025-03-31 15:23:14.338 | DEBUG    | __main__:process_new_data:2 - Processing new data: /DSIA/nlp/bertrend/data/feeds/feed_sobriete/2024-12-30_feed_sobriete.jsonl
2025-03-31 15:23:14.340 | INFO     | bertrend.BERTrend:restore_model:629 - Loading BERTrend model from: /home/jerome/dev/cache/models/sobriete_models
2025-03-31 15:23:14.767 | DEBUG    | bertrend.services.embedding_service:_remote_embed_documents:207 - Computing embeddings...
2025-03-31 15:23:14.768 | DEBUG    | bertrend.services.embedding_client:embed_documents:124 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 15:23:14.768 | DEBUG    | bertrend.services.embedding_client:embed_documents:128 - Computing embeddings on 932 documents using (1) batches...
2025-03-31 15:23:17.564 | DEBUG    | bertrend.services.embedding_client:embed_batch:94 - Computing embeddings...
2025-03-31 15:23:53.343 | DEBUG    | bertrend.services.embedding_client:embed_batch:103 - Computing embeddings done for batch
2

None

2025-03-31 15:23:56.814 | DEBUG    | __main__:process_new_data:2 - Processing new data: /DSIA/nlp/bertrend/data/feeds/feed_sobriete/2025-01-06_feed_sobriete.jsonl
2025-03-31 15:23:56.815 | INFO     | bertrend.BERTrend:restore_model:629 - Loading BERTrend model from: /home/jerome/dev/cache/models/sobriete_models
2025-03-31 15:23:56.981 | DEBUG    | bertrend.services.embedding_service:_remote_embed_documents:207 - Computing embeddings...
2025-03-31 15:23:56.982 | DEBUG    | bertrend.services.embedding_client:embed_documents:124 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 15:23:56.982 | DEBUG    | bertrend.services.embedding_client:embed_documents:128 - Computing embeddings on 825 documents using (1) batches...
2025-03-31 15:23:58.078 | DEBUG    | bertrend.services.embedding_client:embed_batch:94 - Computing embeddings...
2025-03-31 15:24:28.073 | DEBUG    | bertrend.services.embedding_client:embed_batch:103 - Computing embeddings done for batch
2

,timestamp,topic,title,description
0,2025-01-06,2,Bilan des Transports et Réduction des Émission...,Ce thème aborde les initiatives gouvernemental...
1,2025-01-06,5,Soutien à l'Innovation Énergétique pour Tous l...,"Face à l'augmentation des coûts de l'énergie, ..."
2,2025-01-06,7,Gestion des Risques Énergétiques en Hiver,Ce thème aborde les enjeux de la disponibilité...
3,2025-01-06,11,Soutien à l'énergie face au changement climatique,Ce thème explore les initiatives gouvernementa...
4,2025-01-06,12,Soutien à l'Énergie et Flexibilité des Install...,"Face à l'augmentation des coûts énergétiques, ..."
5,2025-01-06,14,Soutien tarifaire face à la crise énergétique,Face à la hausse des coûts de l'énergie depuis...
6,2025-01-06,15,Soutien Énergétique et Festivités à Paris,Ce thème explore l'initiative de l'État frança...
7,2025-01-06,17,Soutien Énergétique : Solidarité et Lutte cont...,"Face à la hausse des coûts de l'énergie, l'Éta..."
8,2025-01-06,19,Actions Régionales pour la Biodiversité et l'É...,Ce thème aborde les initiatives mises en place...
9,2025-01-06,20,Soutien gouvernemental face à la hausse des co...,Face à l'augmentation des coûts de l'énergie d...


2025-03-31 15:26:28.608 | DEBUG    | __main__:process_new_data:2 - Processing new data: /DSIA/nlp/bertrend/data/feeds/feed_sobriete/2025-01-20_feed_sobriete.jsonl
2025-03-31 15:26:28.608 | INFO     | bertrend.BERTrend:restore_model:629 - Loading BERTrend model from: /home/jerome/dev/cache/models/sobriete_models
2025-03-31 15:26:28.829 | DEBUG    | bertrend.services.embedding_service:_remote_embed_documents:207 - Computing embeddings...
2025-03-31 15:26:28.830 | DEBUG    | bertrend.services.embedding_client:embed_documents:124 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2025-03-31 15:26:28.830 | DEBUG    | bertrend.services.embedding_client:embed_documents:128 - Computing embeddings on 947 documents using (1) batches...
2025-03-31 15:26:30.139 | DEBUG    | bertrend.services.embedding_client:embed_batch:94 - Computing embeddings...
2025-03-31 15:27:03.147 | DEBUG    | bertrend.services.embedding_client:embed_batch:103 - Computing embeddings done for batch
2

None